In [6]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
import pandas as pd 

from pyspark.sql.types import IntegerType, DateType
from pyspark.sql import Column

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [7]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-03 15:44:07--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.54MB/s    in 0.2s    

2022-09-03 15:44:07 (4.54 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [9]:

# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
tools_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
tools_df.show(n=50, truncate=False)

+-----------+-----------+--------------+----------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
tools_df.count()

1741100

In [11]:
tools_df.columns

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [12]:
tools_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [13]:
len(tools_df.columns)

15

In [14]:
tools_df = tools_df.withColumn('star_rating',tools_df['star_rating'].cast(IntegerType()))
tools_df = tools_df.withColumn('review_date',tools_df['review_date'].cast(DateType()))
tools_df.printSchema

<bound method DataFrame.printSchema of DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: date]>

In [15]:
tools_df.dropDuplicates()

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: date]

In [16]:
tools_df.dropna()

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: date]

In [17]:
review_id_table = tools_df.select(["review_id","customer_id", "product_id", "product_parent", "review_date"]).distinct()
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3N6LLZ1HW1SJY|   52484548|B005WTP3OW|     973135775| 2015-08-31|
| R5IOJ5R94I0UA|   23970967|B00A32VJ78|     862417946| 2015-08-31|
|R29IQTNO1IUOXF|   18574116|B00GGG0X04|     245471621| 2015-08-31|
| R66H626USRHFF|   31246396|B00829POGQ|     643214126| 2015-08-31|
| RX7DBQV94ANF9|   14470506|B009O94W3E|     398511332| 2015-08-31|
| ROKSM4SSTFV3P|    7591774|B0078YA3HQ|     852751725| 2015-08-31|
|R1IN4J2249DWG0|   29771685|B00002X1ZX|     538992924| 2015-08-31|
|R14JTG1KHD0AU3|   10481592|B001EYU8SA|      67734535| 2015-08-31|
|R12BL74AMPKVW1|    7164456|B0013AXUOY|     633360586| 2015-08-31|
| R9E2SNDYL3OI2|   28068916|B000HZJMFM|     986736723| 2015-08-31|
|  RMZRR1QC6SRF|   32677305|B00HPS4FLA|     295698999| 2015-08-31|
|R239IS1N4147CA|   24714641|B00I9IFG86|     116422502| 2015-08

In [18]:
products= tools_df.select(["product_id","product_title"]).distinct()
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...|
|B000OECQOK|Titan 17005 Stain...|
|B0045W0MD4|The American Red ...|
|B00DQQ9ET8|de.power DP-010AA...|
|B001P30BQE|DEWALT DG5204 Pro...|
|B0002TKID8|Channellock GLS-1...|
|B003O684GO|Tajima AC-500B 3/...|
|B00T96T070|UltraFire WF-68B ...|
|B001445114|Stiletto AG-102 A...|
|B00002X1ZX|Stanley Tools 28-...|
|B0006IE7OQ|Neiko 00561A Heav...|
|B00C8STBPC|ZFE 42Pcs Brass S...|
|B00004SUQN|Malco M2006 Max20...|
|B000TCZUWM|Et18 RS Mizar Ele...|
|B000189RGO|Eklind 13218 Metr...|
|B009SB5RHS|Huntsman® - W20 4...|
|B0002IXQHO|Robert Sorby 67HS...|
|B004Q05TTC|Superior Tool 375...|
|B00012Y38W|Bondhus 20199 Bal...|
|B003YL4954|PORTER-CABLE NS18...|
+----------+--------------------+
only showing top 20 rows



In [19]:
customers = tools_df.groupby("customer_id").agg({"customer_id": "count"})
customers = customers.withColumnRenamed("count(customer_id)", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45978717|             2|
|   43622307|             1|
|     740678|             4|
|   29045703|             1|
|   52484548|             3|
|   42560427|             4|
|   17067926|             1|
|   10093406|             1|
|   44979559|             1|
|   19432125|             1|
|   26079415|             1|
|   29931671|             1|
|   12945150|             3|
|   12036434|             1|
|   14230926|             1|
|   45015535|             1|
|   39064792|             1|
|   20709090|             1|
|   45074906|             1|
|   26767269|             2|
+-----------+--------------+
only showing top 20 rows



In [20]:
products = tools_df.select(["product_id","product_title"]).distinct()
products.show()
products.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000JNNWQ2|IRWIN Tools VISE-...|
|B000OECQOK|Titan 17005 Stain...|
|B0045W0MD4|The American Red ...|
|B00DQQ9ET8|de.power DP-010AA...|
|B001P30BQE|DEWALT DG5204 Pro...|
|B0002TKID8|Channellock GLS-1...|
|B003O684GO|Tajima AC-500B 3/...|
|B00T96T070|UltraFire WF-68B ...|
|B001445114|Stiletto AG-102 A...|
|B00002X1ZX|Stanley Tools 28-...|
|B0006IE7OQ|Neiko 00561A Heav...|
|B00C8STBPC|ZFE 42Pcs Brass S...|
|B00004SUQN|Malco M2006 Max20...|
|B000TCZUWM|Et18 RS Mizar Ele...|
|B000189RGO|Eklind 13218 Metr...|
|B009SB5RHS|Huntsman® - W20 4...|
|B0002IXQHO|Robert Sorby 67HS...|
|B004Q05TTC|Superior Tool 375...|
|B00012Y38W|Bondhus 20199 Bal...|
|B003YL4954|PORTER-CABLE NS18...|
+----------+--------------------+
only showing top 20 rows



183811

In [21]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://postgres.cyu2i14ugi0d.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"<enter_username>", 
          "password": "<enter_password>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)